# Serial Connection Development

In [258]:
import serial
import time
from datetime import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from matplotlib.animation import FuncAnimation
from pynput import keyboard
from scipy.optimize import curve_fit
from typing import Callable
import multiprocessing
from ctypes import c_char

from src.pipeline.measurement_processes import start_measurement_processes

In [257]:
start_measurement_processes()

Figure(1000x1000)


/Users/paulrusing/miniconda3/envs/biosig_env/lib/python3.13/site-packages/matplotlib/animation.py:908: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


Terminating processes...
Cleanup completed


Process SamplingProcess:
